In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install scikit-learn
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [4]:
import pandas as pd

file_path = "drive/MyDrive/Rawdata.csv"

encodings = ['utf-8', 'latin1', 'ISO-8859-1', 'utf-16', 'utf-8-sig']
for enc in encodings:
    try:
        df = pd.read_csv(file_path, encoding=enc, on_bad_lines='skip', low_memory=False)
        print(f"  successfull: {enc}")
        break
    except UnicodeDecodeError:
        print(f" fail: {enc}")
        continue

df.head()

 fail: utf-8
  successfull: latin1


,STARTDATE,ENDDATE,PROGRESS,DURATION,FINISHED,RECORDEDDATE,DISTRIBUTIONCHANNEL,USERLANGUAGE,ARM,CONSENT,...,DEMOGRAPHIC6,DEMOGRAPHIC7,DEMOGRAPHIC8,DEMOGRAPHIC8_13_TEXT,DEMOGRAPHIC9,DEMOGRAPHIC10,DEMOGRAPHIC10_4_TEXT,DEMOGRAPHIC11,DEMOGRAPHIC12,DEMOGRAPHIC13
0,6/3/2020 13:05,6/3/2020 13:11,100,370,True,6/3/2020 13:11,anonymous,EN,1,"Yes, I consent.",...,Yes,No,Roman Catholic (Catholic),NaN,Somewhat important,Male,NaN,"$20,001-$50,000",White,Lower middle class
1,6/3/2020 13:06,6/3/2020 13:13,100,447,True,6/3/2020 13:13,anonymous,EN,0,"Yes, I consent.",...,Yes,No,"Protestant (Baptist, Methodist, Non-denominati...",NaN,Somewhat important,Female,NaN,"$10,001-$20,000",White,Lower middle class
2,6/3/2020 13:07,6/3/2020 13:15,100,478,True,6/3/2020 13:15,anonymous,EN,2,"Yes, I consent.",...,Yes,No,Something else (SPECIFY),No,Don't know/Refused,Female,NaN,"< $10,000",White,Lower class/poor
3,6/3/2020 13:05,6/3/2020 13:15,100,605,True,6/3/2020 13:15,anonymous,EN,2,"Yes, I consent.",...,Yes,No,Roman Catholic (Catholic),NaN,Not too important,Female,NaN,"$20,001-$50,000",White,Lower class/poor
4,6/3/2020 13:11,6/3/2020 13:16,100,275,True,6/3/2020 13:16,anonymous,EN,1,"Yes, I consent.",...,Yes,No,"Protestant (Baptist, Methodist, Non-denominati...",NaN,Not too important,Female,NaN,"< $10,000",White,Lower class/poor


In [7]:
from sklearn.preprocessing import OneHotEncoder

df.columns = df.columns.str.strip()

categorical_columns = [
    "DEMOGRAPHIC13", "DEMOGRAPHIC11", "DEMOGRAPHIC7", "DEMOGRAPHIC3",
    "ELDERLYCARE1", "CHILDCARE3", "CHILDCARE1", "MEDIA17", "MEDIA16"
]


In [8]:

if "MEDIA16" in df.columns:

    df["MEDIA16"] = df["MEDIA16"].astype(str).fillna("No Response")


    media16_options = [
        "Yes, from a soup kitchen",
        "Yes, from a school district",
        "Yes, from the food stamps program/EBT",
        "Yes, from a community food distribution program",
        "No, have not sought food assistance",
        "Don’t know"
    ]

    for option in media16_options:
        df[f"MEDIA16_{option}"] = df["MEDIA16"].apply(lambda x: 1 if option in x else 0)


    categorical_columns.remove("MEDIA16")


In [9]:

df_categorical = df[categorical_columns]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_array = encoder.fit_transform(df_categorical)

encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_columns))

df_final = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)

df_final.head()


,STARTDATE,ENDDATE,PROGRESS,DURATION,FINISHED,RECORDEDDATE,DISTRIBUTIONCHANNEL,USERLANGUAGE,ARM,CONSENT,...,DEMOGRAPHIC3_2008,DEMOGRAPHIC3_2010,ELDERLYCARE1_No,ELDERLYCARE1_Yes,CHILDCARE3_No,CHILDCARE3_Yes,CHILDCARE1_No,CHILDCARE1_Yes,MEDIA17_No,MEDIA17_Yes
0,6/3/2020 13:05,6/3/2020 13:11,100,370,True,6/3/2020 13:11,anonymous,EN,1,"Yes, I consent.",...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,6/3/2020 13:06,6/3/2020 13:13,100,447,True,6/3/2020 13:13,anonymous,EN,0,"Yes, I consent.",...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,6/3/2020 13:07,6/3/2020 13:15,100,478,True,6/3/2020 13:15,anonymous,EN,2,"Yes, I consent.",...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,6/3/2020 13:05,6/3/2020 13:15,100,605,True,6/3/2020 13:15,anonymous,EN,2,"Yes, I consent.",...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,6/3/2020 13:11,6/3/2020 13:16,100,275,True,6/3/2020 13:16,anonymous,EN,1,"Yes, I consent.",...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [10]:

df_encoded_only = pd.concat([encoded_df, df[[col for col in df.columns if "MEDIA16_" in col]]], axis=1)

encoded_only_file_path = "/content/drive/MyDrive/Vedika_onehotencoded.csv"

df_encoded_only.to_csv(encoded_only_file_path, index=False)

